#### Deep Learning
#### Autores: Mariana David 
#### Autores: Pablo Escobar 
#### Laboratorio 6 y 7

# Red neuronal basado en contenido

In [1]:
#Importaciones 

import tensorflow as tf
from scipy.sparse import csr_matrix
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [2]:
from scipy.sparse import csr_matrix


def configure_tpu():
    # Compruebar entorno
    tpu_found = False
    try:
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver()  # Intenta resolver automáticamente
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        tpu_found = True
    except Exception as e:
        print("No se encontró una TPU. Usando CPU/GPU en su lugar.")
    
    if tpu_found:
        return tf.distribute.experimental.TPUStrategy(resolver)
    else:
        return tf.distribute.OneDeviceStrategy("/device:CPU:0")  # Usa CPU si no se encuentra una TPU

strategy = configure_tpu()

with strategy.scope():
    # Cargando datos y preprocesamiento (usando el 10% de los datos)
    data = pd.read_csv('archive/Books.csv')
    data = data.sample(frac=0.1, random_state=1).reset_index(drop=True)

    # Creación de perfiles de contenido
    data['Content'] = data['Book-Title'] + ' ' + data['Book-Author'].astype(str) + ' ' + data['Year-Of-Publication'].astype(str) + ' ' + data['Publisher']
    data['Content'].fillna('ValorPredeterminado', inplace=True)

    print(data["Book-Title"])

    # Vectorización de texto (TF-IDF)
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(data['Content'])

    # Convierte tfidf_matrix en una matriz dispersa
    tfidf_matrix = csr_matrix(tfidf_matrix)

    # Cálculo de similitud de texto (similitud coseno)
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

    # Generación de recomendaciones
    def get_recommendations(book_title, cosine_sim=cosine_sim):
        if book_title in data['Book-Title'].values:
            idx = data[data['Book-Title'] == book_title].index[0]
            if idx < len(cosine_sim):
                sim_scores = list(enumerate(cosine_sim[idx]))
                sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
                sim_scores = sim_scores[1:11]
                book_indices = [i[0] for i in sim_scores]
                return data['Book-Title'].iloc[book_indices]
            else:
                return pd.Series([])
        else:
            return pd.Series([])


No se encontró una TPU. Usando CPU/GPU en su lugar.


C:\Users\Mariana\AppData\Local\Temp\ipykernel_12364\955398776.py:24: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('archive/Books.csv')


0                                       LOST IN THE MUSEUM
1                                   Skinned Alive: Stories
2         Silly Mid-off: How to Survive the Cricket Season
3                        El Pianista del Gueto de Varsovia
4        Merry Wives of Windsor (Arden Shakespeare Seco...
                               ...                        
27131    Valley of Horses (Thorndike Large Print Basic ...
27132    An Elizabethan Progress: The Queen's Journey i...
27133    Millionaire'S Pregnant Bride (Texas Cattleman'...
27134                                             Rosewood
27135     Data &amp; Computer Communications (6th Edition)
Name: Book-Title, Length: 27136, dtype: object


In [3]:
libro = "Skinned Alive: Stories"

if libro in data['Book-Title'].values:
    recommendations = get_recommendations(libro)
    print("La recomendación es:")
    print(recommendations)
else:
  print(f"'{libro}' no se encuentra en el conjunto de datos.")

La recomendación es:
4788                      Fanny: A Fiction (White, Edmund)
17816                                     Forgetting Elena
5008                                  Sketches from Memory
6696                    The Enchanted Land (Romance Alive)
26573      To the Wedding: A Novel (Vintage International)
19116                                          Another You
470                        Our Paris: Sketches from Memory
8533          River Dogs: Stories (Vintage Contemporaries)
18308    The Safety of Objects: Stories (Vintage Contem...
10586                      Nightrose (Romance Alive Audio)
Name: Book-Title, dtype: object
